# 텔레그램 봇과 CCTV
* 난이도 : ★★★★☆☆☆☆☆☆
* 필요라이브러리: numpy, openCV, scikit-image, pillow, telepot, logging, threading, time


* 기존의 텔레그램 봇 기능과 CCTV 기능을 합쳐서 이미지 변화가 생기면 텔레그램으로 알려주는 프로그램을 만듭니다.
* 텔레그램에서 캡쳐 명령이 오면 현재 화면을 전송하는 기능도 추가 합니다.

In [ ]:
# 텔레그램봇을 위한 라이브러리
import telepot

# 로그 관련 라이브러리
import logging

# 로그 생성
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

import cv2
import numpy as np
from skimage.measure import compare_ssim
from PIL import ImageFont, ImageDraw, Image

import threading
import time 

# 텔레그램 명령으로 캡쳐를 요청한 경우 한 프레임만 텔레그램으로 전송하기 위해 사용하는 변수
# 웹캠 디바이스는 메인 쓰레드에서 이미 오픈해서 구동중이기 때문에
# 이를 따로 함수로 구현하지 않고 텔레그램 명령이 들어오면 send_frame = True로 변경하고
# 메인 쓰레드에서는 send_frame == True 인 상태면 그냥 캡쳐를 전송하게 로직을 구현했습니다.
send_frame = False

# 감시 쓰레드 동작 설정 변수
run_thread = False

# 봇생성 후 받은 텔레그램 API 토큰
# 본인이 생성한 텔레그램 API 토큰을 작성하세요
TELEGRAM_TOKEN = "646258683:AAG1Q5_pXsThCvi2J0de_J--plCJpGYDqj0"



def make_text(img, text):
    '''이미지 우하단에 텍스트를 작성하는 함수
    opencv 에서 제공하는 텍스트는 기능이 너무 제한적이라 pillow 로 텍스트를 작성합니다.
    Args:
        img (openc_cv) : opencv 이미지
        text (str) : 텍스트
    
    Returns:
        opencv : 텍스트가 작성된
    '''

    # 폰트명과 사이즈를 인자로 넘겨줘 PIL 의 ImageFont 객체 생성
    font = ImageFont.truetype("malgun.ttf", 17)
    
    # 해당 텍스트의 영역을 구합니다.
    text_w, text_h = font.getsize(text)

    # 이미지의 w, h 
    w = img.shape[1]
    h = img.shape[0]

    # 텍스트가 실제 이미지에 위치할 위치값 계산
    X_POS = w - text_w - 10
    Y_POS = h - text_h - 10
    
    # 이미지의 텍스트 영역에 투명 사각형을 그리기 위해서는 오버레이 처리를 해야합니다.
    # 먼저 원본 이미지의 복사본을 생성
    overlay = img.copy()
    
    # 복사본 이미지의 텍스트 영역에 사각형을 Fill 형태로 채웁니다.
    cv2.rectangle(overlay, (X_POS - 3, Y_POS), (X_POS + text_w, Y_POS + text_h + 3), (0, 0, 0), -1)

    # 사각형의 투명도를 설정합니다.
    alpha = 0.5 
    
    # 원본이미지와 복사본 이미지를 합칩니다.
    # addWeighted 함수는 이미지를 합칠시에 가중치를 주어 합치는 방식인데
    # 여기서는 alpha 값만큼을 1-alpha 값만큼으로 합치게 되니 실제 투명박스 부분만 합쳐지게 됩니다. 
    img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

    # opencv -> pillow
    img_pil = Image.fromarray(img)

    # pillow 이미지로 draw 객체는 생성합니다.
    draw = ImageDraw.Draw(img_pil)
    
    # 이미지에 텍스트를 작성합니다.
    draw.text((X_POS, Y_POS), text, (255,255,255), font=font)

    # pillow -> opencv    
    img = np.array(img_pil)

    return img

def send_frame_to_telegram(chat_id, frame):
    '''텔레그램으로 이미지를 전송할 함수
    
    Args:
        chat_id (int) : 챗 아이디
        frame (opencv) : 프레임 이미지
    '''

    # 텔레그램봇의 전송 명령이 파일 데이터를 직접 보낼수가 없어서
    # 파일로 저장 후 해당 파일을 전송하는 형태로 구현되었습니다.
    with open("_tmp.jpg", "wb") as file, open("_tmp.jpg", "rb") as read:
        # 넘어온 frame 데이터를 jpg 형태로 인코딩 후 파일로 저장 후
        file.write(np.array(cv2.imencode('.jpg', frame)[1]).tostring())
        # 텔레그램으로 전송
        bot.sendPhoto(chat_id, read)
        
def capture_video(chat_id):
    '''메인 함수'''

    # 캡쳐 전송 명령을 처리할 send_frame 전역 변수와
    # 쓰레드 상태를 설정할 run_thread 전역 변수
    global send_frame, run_thread

    # 웹캠 캡쳐 오픈
    cap = cv2.VideoCapture(0)

    # 웹캠 사용가능한지 체크
    if (cap.isOpened() == False): 
        print("카메라를 오픈할 수 없습니다.")
    
    # 연결된 웹캠 디바이스의 프레임 해상도를 알아옵니다.
    # float 를 int 로 캐스팅 해야 합니다.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    # 웹캠의 내용을 파일로 저장하기 위해 VideoWriter 를 설정합니다.
    # 여기서 'M', 'J', 'P', 'G' 는 MotionJPEG 코덱을 사용한다는 이야기인데
    # D I V X 같은 코덱도 해당 컴퓨터에 설치 되어있으면 사용할 수 있습니다.
    out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

    # 이미지 변화를 감지하기 위해 저장해놓을 이전 이미지 변수
    old_image = None

    # 화면 변화 감지시 빨간색 박스를 그리기 위해 (BGR)
    c = (0, 0, 255)

    # 텔레그램으로 이미지를 마지막 보낸 시간 저장
    last_send_time = 0

    # 무한루프의 동작 여부는 전역 변수인 run_thread 에 설정 됩니다.
    while run_thread:
        # 웹캠 장치에서 프레임을 읽어옵니다.
        ret, frame = cap.read()
    
        # 프레임 읽기에 성공하면
        if ret == True: 
            # 파일로 저장합니다.
            out.write(frame)

            # 이전 이미지가 존재한다면
            if old_image is not None:
                # 현재 이미지와 이전 이미지를 흑백으로 변환합니다.
                grayA = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                grayB = cv2.cvtColor(old_image, cv2.COLOR_BGR2GRAY)

                # 사이킷 이미지의 compare_ssim 함수를 사용하여
                # 이미지의 유사도를 측정합니다.
                (score, diff) = compare_ssim(grayA, grayB, full=True)
                diff = (diff * 255).astype("uint8")
                #print("SSIM: {}".format(score))


                # 현재 프레임의 width, height 을 구합니다.
                w = frame.shape[1]
                h = frame.shape[0]

                # 이미지에 유사도를 출력
                frame = make_text(frame, "유사도: {:.12f}".format(score))

                # 이미지 유사도가 xxx이하면
                if score < 0.90:
                    # 이미지에 6 두께의 빨간 사각형을 그립니다.
                    cv2.rectangle(frame, (0, 0), (w, h), c, 20)

                    # 이미지를 최소 10초 안에 재전송하지 않게 합니다.
                    if last_send_time == 0 or time.time() - last_send_time > 10:
                        send_frame_to_telegram(chat_id, frame)

                    # 마지막 보낸 시간 갱신
                    last_send_time = time.time()

                # 텔레그램으로 부터 캡쳐 명령이 오면
                if send_frame:
                    # 현재 이미지를 텔레그램으로 전송합니다.
                    send_frame_to_telegram(chat_id, frame)
                    # 캡쳐 명령 처리 변수는 False 로 초기화 합니다.
                    # 이걸 안하면 계속 계속 계속 보내겠죠.....
                    send_frame = False

            # 다음 프레임 비교를 위해 현재 프레임을 이전 이미지 변수에 저장
            old_image = frame

            # 이미지 출력
            cv2.imshow('CCTV',frame)

            # Q 키 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        # 프레임 읽기 실패시 종료
        else:
            break 
    
    # 웹캠 장치와 파일저장을 모두 종료시킵니다.
    cap.release()
    out.release()
    
    # opencv 로 생성된 창을 모두 삭제 합니다.
    cv2.destroyAllWindows() 


def handler(msg):
    global send_frame, run_thread
    '''텔레그램 봇의 메시지 이벤트가 발생되면 콜백호출 되는 함수

    Args:
        msg (object) : 텔레그램봇의 데이터가 딕셔너리 형태로 넘어옵니다.

    Returns:
        Nothing
    '''

    # glance() 함수는 는 telepot 의 텔레그램 메세지객체 에서 헤드라인 정보만 추출해서 리턴하는 함수 입니다.
    content_type, chat_type, chat_id, msg_date, msg_id = telepot.glance(msg, long=True)

    # 텍스트 타입인 경우
    if content_type == "text":
        # 채팅으로 입력된 문자열
        str_message = msg["text"]
        # 채팅 시작이 '/' 문자인 경우만 명령어로 인식 합니다.
        if str_message[0:1] == "/":
            # 명령어인경우 공백으로 분리 합니다. 
            # ex) /mon start 이런경우 mon 가 명령이 되고 start 는 인자로 처리 하기 위함
            args = str_message.split(" ")

            # 0번째가 명령어
            # ['/mon', 'start']
            command = args[0]

            # 리스트에서 명령어는 지웁니다.
            # start, stop 만 0번째에 위치하게 됩니다.
            # ['start']
            del args[0]

            if command == "/mon":
                # 감시시작 명령이 오면
                if args[0] == "start":
                    # 현재 쓰레드가 동작중이지 않으면
                    if not run_thread:
                        # 동작 설정
                        run_thread = True
                        # 쓰레드 생성 및 시작
                        t = threading.Thread(target=capture_video, args=(chat_id,))
                        t.daemon = True
                        t.start()
                        bot.sendMessage(chat_id, "현재 감시를 시작했습니다.")
                        print("감시 시작")
                    else:
                        # 동작중이라고 텔레그램으로 알림
                        bot.sendMessage(chat_id, "현재 감시가 동작중입니다.")
                        print("감시 중..")
                # 감시 중지 명령이 오면
                elif args[0] == "stop":
                    # 쓰레드가 동작중이면
                    if run_thread:
                        print("감시 종료")
                        bot.sendMessage(chat_id, "감시를 종료했습니다.")
                        run_thread = False
            # 캡쳐 명령이 오면
            elif command == "/c":
                # 캡쳐 변수를 True 설정
                # 나머지는 쓰레드 내에서 알아서 처리되게..
                send_frame = True


if __name__ == "__main__":
    bot = telepot.Bot(TELEGRAM_TOKEN)
    bot.message_loop(handler, run_forever=True)
    
